# Implementación Path Relinking
---

In [65]:
from structure import instance, solution
from algorithms import grasp
from structure.solution import evaluate
from copy import deepcopy
import pandas as pd
import random

In [66]:
path = "instances/MDG-a_12_100_m10.txt"
inst = instance.readInstance(path)
sol, conjunto_soluciones, tiempo_final_grasp = grasp.execute(inst, 1, 0.1)

In [67]:
conjunto_soluciones_pd = pd.DataFrame(conjunto_soluciones)
print(len(conjunto_soluciones_pd))
conjunto_soluciones_pd = conjunto_soluciones_pd.drop_duplicates("sol")

775


In [68]:
conjunto_soluciones_pd.sort_values("of", ascending=False)

,instance,sol,of
10,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{64, 96, 13, 15, 51, 20, 86, 55, 59, 93}",354.25
75,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{2, 98, 37, 90, 91, 12, 20, 26, 59, 63}",351.82
126,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{64, 70, 43, 79, 16, 20, 86, 59, 60, 93}",350.18
8,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{5, 70, 39, 8, 74, 43, 14, 79, 17, 60}",349.53
3,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{66, 3, 78, 14, 15, 83, 53, 56, 61, 94}",349.45
...,...,...,...
209,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{1, 97, 6, 77, 80, 18, 52, 26, 61, 62}",317.90
418,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{1, 97, 80, 18, 83, 20, 21, 87, 61, 95}",315.60
140,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{66, 35, 38, 39, 75, 13, 48, 17, 19, 58}",315.28
397,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{0, 9, 43, 78, 14, 79, 17, 24, 61, 94}",315.16


In [69]:
sol_max = conjunto_soluciones_pd[conjunto_soluciones_pd["of"] == max(conjunto_soluciones_pd["of"])].iloc[0,1]
conjunto_soluciones_pd["n_distintos"] = conjunto_soluciones_pd["sol"].map(lambda x: len(x.difference(sol_max)))
conjunto_soluciones_pd[conjunto_soluciones_pd["n_distintos"] > 0].sort_values("of", ascending=False)

,instance,sol,of,n_distintos
75,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{2, 98, 37, 90, 91, 12, 20, 26, 59, 63}",351.82,8
126,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{64, 70, 43, 79, 16, 20, 86, 59, 60, 93}",350.18,5
8,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{5, 70, 39, 8, 74, 43, 14, 79, 17, 60}",349.53,10
3,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{66, 3, 78, 14, 15, 83, 53, 56, 61, 94}",349.45,9
241,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{2, 98, 37, 60, 15, 16, 20, 86, 59, 93}",349.36,5
...,...,...,...,...
209,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{1, 97, 6, 77, 80, 18, 52, 26, 61, 62}",317.90,10
418,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{1, 97, 80, 18, 83, 20, 21, 87, 61, 95}",315.60,9
140,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{66, 35, 38, 39, 75, 13, 48, 17, 19, 58}",315.28,9
397,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{0, 9, 43, 78, 14, 79, 17, 24, 61, 94}",315.16,10


A continuación, se realiza una implementación recursiva de Path Relinking entre dos soluciones específicas.

In [70]:
sol_0 = conjunto_soluciones[14]
sol_1 = conjunto_soluciones[57]

In [71]:
def transicion(sol_0, sol_1):
    best = None
    resultado = 0
    sol_2 = deepcopy(sol_0)
    sol_guia = deepcopy(sol_1)
    sol_guia["sol"] = sol_guia["sol"].difference(sol_0["sol"])
    for s0 in sol_0["sol"]:
        for s1 in sol_guia["sol"]:
            if s1 not in sol_0["sol"]:
                sol_2["sol"].remove(s0)
                sol_2["sol"].add(s1)
                sol_2["of"] = evaluate(sol_2)
                if best is None or sol_2["of"] > best["of"]:
                    anyadido = s1
                    best = deepcopy(sol_2)
                sol_2["sol"].remove(s1)
                sol_2["sol"].add(s0)
    
    sol_guia["sol"].remove(anyadido)

    if len(sol_guia["sol"]) > 1:
        resultado = transicion(best, sol_guia)

        if resultado["of"] < best["of"]:
            resultado = best
    else:
        print(best["sol"])
        resultado = best
    

    return resultado

best = transicion(sol_0, sol_1)

{66, 3, 11, 83, 53, 56, 92, 61, 94, 63}


In [72]:
best["of"]

337.9800000000001

In [73]:
sol_0["of"]

330.04

In [74]:
sol_1["sol"]

{3, 10, 14, 53, 56, 61, 63, 66, 83, 94}

### Opción para el Path Relinking

La siguiente función es la versión final y general de PR paso a paso.

In [75]:
def relink(inicio, guia):
    current = deepcopy(inicio)
    best = deepcopy(inicio)

    to_add = guia["sol"] - current["sol"]
    to_remove = current["sol"] - guia["sol"]

    while to_add:
        vecinos = []
        for a in to_add:
            for r in to_remove:
                nuevo = deepcopy(current)
                nuevo["sol"].remove(r)
                nuevo["sol"].add(a)
                nuevo["of"] = evaluate(nuevo)
                vecinos.append(nuevo)
        if not vecinos:
            break
        current = max(vecinos, key=lambda s: s["of"])
        if current["of"] > best["of"]:
            best = deepcopy(current)

        to_add = guia["sol"] - current["sol"]
        to_remove = current["sol"] - guia["sol"]

    return best


Ahora se busca ejecutar PR entre todos los pares del conjunto élite. Mostramos cuáles son las mejores soluciones sin repetir y la mejor que encuentra.

In [78]:
# Seleccionamos las x mejores soluciones distintas (se puede modificar el nº de soluciones que se toman)
n_soluciones = 10
elite_df = conjunto_soluciones_pd.sort_values("of", ascending=False).drop_duplicates("sol").head(n_soluciones)
elite_solutions = elite_df.to_dict("records")

# Aplicamos Path Relinking entre todos los pares de soluciones
soluciones_pr = []
mejor_pr = None
for i in range(len(elite_solutions)):
    for j in range(i+1, len(elite_solutions)):
        s1 = elite_solutions[i]
        s2 = elite_solutions[j]
        pr1 = relink(s1, s2)
        pr2 = relink(s2, s1)
        soluciones_pr.extend([pr1, pr2])
        for cand in [pr1, pr2]:
            if mejor_pr is None or cand["of"] > mejor_pr["of"]:
                mejor_pr = cand

# Mostramos la mejor solución obtenida por Path Relinking
print("Mejor solución encontrada por Path Relinking:")
print("Conjunto:", mejor_pr["sol"])
print("Valor objetivo:", mejor_pr["of"])

# Convertimos a DataFrame para análisis posterior
df_pr = pd.DataFrame(soluciones_pr).drop_duplicates(subset="sol")
print("\nTop 5 soluciones PR:")
print(df_pr.sort_values("of", ascending=False).head(5))

print(f"\nTotal de soluciones generadas mediante PR: {len(df_pr)}")


Mejor solución encontrada por Path Relinking:
Conjunto: {64, 96, 13, 15, 51, 20, 86, 55, 59, 93}
Valor objetivo: 354.24999999999994

Top 5 soluciones PR:
                                             instance  \
0   {'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...   
18  {'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...   
34  {'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...   
48  {'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...   
60  {'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...   

                                         sol      of  n_distintos  
0   {64, 96, 13, 15, 51, 20, 86, 55, 59, 93}  354.25            0  
18   {2, 98, 26, 37, 59, 12, 20, 90, 91, 63}  351.82            8  
34  {64, 70, 43, 79, 16, 20, 86, 59, 60, 93}  350.18            5  
48    {5, 70, 39, 8, 74, 43, 14, 79, 17, 60}  349.53           10  
60   {66, 3, 78, 15, 14, 83, 53, 56, 61, 94}  349.45            9  

Total de soluciones generadas mediante PR: 9
